# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This problem is a binary (two-class) classification problem. This is because the goal is to identify or categorize students as ones who "need intervention" and ones who "don't need intervention". We are essentially categorizing each data instance (student information) into discrete bins - specifically 2 bins in this case. Since discrete labels are involved, this is an example of supervised classification.
Regression is generally used when the output is a continuous variable.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [88]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [89]:
# TODO: Calculate number of students
n_students = len(student_data.index)

# TODO: Calculate number of features
n_features = len(student_data.columns) - 1 
#Subtract one as the last column is the target variable and is hence not a feature

# TODO: Calculate passing students
n_passed = sum(student_data['passed']=='yes')

# TODO: Calculate failing students
n_failed = sum(student_data['passed']=='no')
#### Alternatively, we can also do n_failed = n_students - n_passed ###

# TODO: Calculate graduation rate
grad_rate = n_passed/float(n_students)*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [90]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [91]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [92]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train
testSize = num_test/float(n_students)
# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all,y_all,test_size = testSize,random_state = 1,stratify=y_all)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

print "Grad rate of the training set: {:.2f}%".format(100 * (y_train == 'yes').mean())
print "Grad rate of the testing set: {:.2f}%".format(100 * (y_test == 'yes').mean())
print "Achieved stratified sampling.!"

Training set has 300 samples.
Testing set has 95 samples.
Grad rate of the training set: 67.00%
Grad rate of the testing set: 67.37%
Achieved stratified sampling.!


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: ** 

Three supervised classification methods that are appropriate for this problem are as follows.
1. Support Vector Machines (SVM)
2. Decision Trees
3. Gaussian Naive Bayes

SVMs are generally used for two-class classification problems. They are used when the data is linearly separable in some dimension. Linear separability can be induced by using Kernel functions which increase the dimensionality of the data and if the kernel function is appropriate, would result in the data being linearly separable in the higher dimensional space. The major advantage of SVMs lies in the fact that most data can be made at least reasonably linearly separable using an appropriate kernel function. This means that SVMs can be applied to a large variety of problems. The weakness of SVM is that training is cubic in the size of the input O(n^3). This means that if the dataset is large, SVMs take a very long time to train. They also don't work very well when the data is too noisy. For this problem, the dataset is not huge - there are only 300 training instances which means that we can expect good performance out of the SVM classifier without an excessive training time.

Decision Trees are good for a variety of classification problems. Decision trees separate the data space into sub-regions across various dimensions to separate out one class from another. Complex decision boundaries can also be obtained from decision trees. A key advantage of decision trees is that they are easy to interpret. They basically implement an if-then-else condition in each node which is understandable to humans unlike other classification algorithms which produce results in an abstract space. As mentioned earlier, highly complex decision boundaries can also be obtained if required. Decision trees also tend to perform well for categorical or discrete input features. The weakness of Decision trees is that they are prone to overfitting. Splitting in decision trees occurs with respect to maximum information gain and this depends highly on the data. Hence, there is quite a high dependence on the data and decision trees might find it hard to generalize well. In this dataset, most of the features are discrete and hence decision trees can be expected to perform well.

Naive Bayes classifiers generally work well when the input features are independent of each other. This is because the algorithm makes this assumption of independent features. This classifier is very simple and the training time is linear in the number of inputs O(n). Hence, training is quite fast. The weakness of the Naive Bayes classifier is that it tends to perform poorly for inputs with features that are dependent on each other which happens quite often in real datasets. In this dataset however, there seems to be many features that intuitively seem independent like school, sex, age, address, famsize, Medu, Fedu and so on. The school that the student studies in, intuitively seems independent of the student's gender, age, address, Mother's education, Father's education and so on. Similarly, there seems to be independence between most of the variables with one another. Hence, using Naive Bayes to classify this dataset seems appropriate.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [93]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [94]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# TODO: Initialize the three models
clf_A = SVC(random_state=1)
clf_B = DecisionTreeClassifier(random_state=1)
clf_C = GaussianNB()

# TODO: Set up the training set sizes
X_train_100 = X_train[0:100]
y_train_100 = y_train[0:100]

X_train_200 = X_train[0:200]
y_train_200 = y_train[0:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C]:
    print "\n{}: \n".format(clf.__class__.__name__)
    for n in [100, 200, 300]:
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)


SVC: 

Training a SVC using a training set size of 100. . .
Trained model in 0.0055 seconds
Made predictions in 0.0031 seconds.
F1 score for training set: 0.8707.
Made predictions in 0.0029 seconds.
F1 score for test set: 0.8000.
Training a SVC using a training set size of 200. . .
Trained model in 0.0096 seconds
Made predictions in 0.0085 seconds.
F1 score for training set: 0.8491.
Made predictions in 0.0038 seconds.
F1 score for test set: 0.8026.
Training a SVC using a training set size of 300. . .
Trained model in 0.0161 seconds
Made predictions in 0.0118 seconds.
F1 score for training set: 0.8701.
Made predictions in 0.0036 seconds.
F1 score for test set: 0.8026.

DecisionTreeClassifier: 

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0015 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.6050.
Training a DecisionTreeClassifier using a training set

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Support Vector Machines (SVM)**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------:|
| 100               |   0.0051                |  0.0015                |   0.8707         |  0.8000        |
| 200               |        0.0051           |  0.0019                |   0.8491         |  0.8026        |
| 300               |        0.0099           |  0.0026                |   0.8701         |    0.8026      |

** Classifer 2 - Decision Trees (DT)**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0015              |    0.0004              |  1.0000          |     0.6050      |
| 200               |     0.0022              |    0.0003              |  1.0000          |     0.6393      |
| 300               |        0.0027           |   0.0003               |  1.0000          |     0.7391      |

** Classifer 3 - Gaussian Naive Bayes**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0010             |    0.0004              |    0.7966        |  0.6379         |
| 200               |      0.0010             |    0.0005              |    0.8148        |  0.7344         |
| 300               |      0.0017             |    0.0005              |    0.8190        |  0.7500         |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Based on the results tabulated above, we see that the F1 test scores for the Support Vector Machines (SVM) algorithm are the highest. The decision tree algorithm performs quite poorly because of overfitting. It is evident that the decision tree is overfitting from the F1 training scores for the decision trees which are all 1.00. This means that the decision tree has come up with a solution that fits the data exactly. This overfitting may be reduced by adjusting parameters such as min_samples_split and others. The Gaussian Naive Bayes performs similar to the decision trees in terms of F1 test scores. This may be because of the fact that not all the features are independent of each other. There may be a few features which are actually dependent resulting in lower F1 test scores. For example, travel time and study time may be related with some degree of inverse proportionality. Similarly health and absences may also be dependent.

Hence, we can infer that the SVM is the best model to pursue given that its F1 test score is the highest and also, the F1 training and testing scores are quite similar while for the other two algorithms, the training score is quite high while the test score is low indicating overfitting. The only challenge with the SVM is that the training time is theoritically large. However, for a small dataset of 300 training instances, this should not cause significant deterioration in speed of training. Hence we expect that the SVM, if tuned appropriately with optimal parameters can result in optimal classifiication performance based on the available data and resources.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **

The Support Vector Machine algorithm draws a linear decision boundary (straight lines or planes or hyperplanes) to separate the data into two classes. In the case of our dataset, these two classes are either the student "needs intervention" or "does not need intervention". The algorithm computes the position and orientation of the linear separator based on a notion of maximizing the margin. Margin can be defined as the distance between the linear separator and the closest data point in either of the two classes. Hence maximizing the margin means maximizing the distance between the linear separator and the data points of either class. 

![](images/svm.png)

Relating the image to our dataset, if the red squares are students who need intervention and the blue circles are those who don't, we see that the line that separates the data with the highest margin is the solid green line marked as "Original hyperplane". In a sense, the algorithm requires fewer training data as only the data near the decision boundary matter. Hence, even though our dataset contains just 300 data instances, SVM seems to be performing a good job. The filled squares and circles are the data instances that are closest to the hyperplane and hence, they are the ones that matter to the SVM algorithm. These data instances are called "Support Vectors".


The fact that a linear separator is used implies that the SVMs can be used only if the data at hand is linearly separable. However, this isn't the case very often. In cases where the data is not inherently linearly separable, we use a "kernel function". Such kernel functions add some dimensionality to the data that makes the data linearly separable in that higher dimensional space. For example, if our dataset which has 30 dimensions (columns representing the features) is not linearly separable in 30 dimensions, we can use kernel functions like "Radial Basis Function" (rbf) which adds more dimensions such that the data in that higher dimensional space is linearly separable. Once the data is made linearly separable, the basic SVM algorithm is then run on the higher dimensional data which is now linearly separable and SVM finds the linear separator in that higher dimensional space. SVM then uses this linear separator to predict which class any future instance belongs to. For example, in the future, if we give a student's data to the model, the model will be able to predict whether the student requires intervention or not. There are several kernel functions that can be used and even if one kernel function does not work as expected, others can be tried out and the best kernel function for a dataset can be determined.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [95]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from matplotlib import pyplot as plt
#from sklearn.preprocessing import Normalizer

#Normalize the input training and test features

####################################################################################################
# TRIED NORMALIZING. F1 TEST SCORE SEEMS TO REDUCE TO 0.80 AFTER NORMALIZING. THAT'S WHY COMMENTED #
####################################################################################################

#normer = Normalizer()
#X_train = normer.fit_transform(X_train)
#X_test = normer.fit_transform(X_test)

# TODO: Create the parameters list you wish to tune
C_range = [0.1,1., 10., 15., 20.]
gamma_range = [1/2000., 1/1000., 1/100., 1/20., 1/10.]
parameters = {'C': C_range, 'gamma': gamma_range}

# TODO: Initialize the classifier
clf = SVC(random_state=1)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,parameters,f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

print grid_obj.best_params_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

scores = [x[1] for x in grid_obj.grid_scores_]
scores = np.array(scores).reshape(len(C_range), len(gamma_range))
plt.figure(figsize=(8, 6))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores, interpolation='nearest', cmap='YlGnBu')
plt.xlabel('gamma')
plt.ylabel('C')
plt.colorbar()
plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
plt.yticks(np.arange(len(C_range)), C_range)
plt.title(
    "The best parameters are {} with a score of {:0.2f}.".format(
   grid_obj.best_params_, grid_obj.best_score_)
)
plt.show()

{'C': 1.0, 'gamma': 0.05}
Made predictions in 0.0089 seconds.
Tuned model has a training F1 score of 0.9284.
Made predictions in 0.0029 seconds.
Tuned model has a testing F1 score of 0.8289.


<img src="images/heatmap_GridSearch.png">

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final model's F1 test score is 0.8289 which is higher than the F1 test score with default parameters, 0.8026. The parameter that was changed here is gamma = 0.05 from the default gamma = 1/(#features) = 1/30 in our case. However, there is a significant improvement in the training F1 score - 0.9284 with gamma = 0.05 as compared to 0.8701 in the default parameter case. Such a high training score may suggest over-fitting; however the test scores seem to be slightly higher and hence, we accept this model (with parameters as kernel = 'rbf', C = 1 and gamma = 0.05) as the optimal one.

Hence, using this model, we can predict whether a student needs intervention or not with an accuracy of approximately 83%.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.